## Generate database of steady state images

In [7]:
import os
import pickle
from itertools import product
from contextlib import suppress
import sys; sys.path.insert(0, "../..")

import numpy as np
from tqdm.auto import tqdm

import groovy.pde

In [8]:
# Create directory to store all the data
folder = 'database'
with suppress(FileExistsError):
    os.mkdir(folder)

In [9]:
# Create a parameter grid
size = 1
ru_grid = np.linspace(0, 1, size)
rv_grid = np.linspace(0, 1, size)
f_grid  = np.linspace(0.01, 0.1, size)
k_grid  = np.linspace(0.045, 0.07, size)

In [10]:
# Run all the simulations
frame_t0 = groovy.pde.initialframe((100, 100), conctype='clump')

for i, (ru, rv, f, k) in enumerate(tqdm(product(ru_grid, rv_grid, f_grid, k_grid),
                                        desc='Params', leave=True, total=size**4)):
    time_array = groovy.pde.evolve(frame_t0, ru, rv, f, k, boundary='wrap')
    final_frame = time_array[:,:,-1,:]

    # Pickle the results
    with open(f"{folder}/{ru}_{rv}_{f}_{k}.pkl", 'wb') as f:
        pickle.dump(final_frame, f)

In [6]:
print('Expected number of files:', size**4)
print('Actual number of files:', len(os.listdir(folder)))

Expected number of files: 81
Actual number of files: 81
